In [59]:
import sys
import json

params = {}
default = {'training_folder': '../../scenarios/test', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)
num_cores=4

{'training_folder': '../../scenarios/test', 'params': {}}


In [60]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from shapely.geometry import LineString
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from quetzal.io.gtfs_reader.importer import get_epsg

In [61]:
base_folder = argv['training_folder']
pt_folder = base_folder + '/inputs/pt/'
road_folder = base_folder + '/inputs/road/'
input_folder = base_folder +'/inputs/'
od_folder = base_folder + '/inputs/od/'
output_folder = base_folder +'/outputs/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# intputs

In [62]:
links = gpd.read_file(pt_folder + 'links.geojson') 
nodes = gpd.read_file(pt_folder + 'nodes.geojson')
links = links.set_index('index')
nodes = nodes.set_index('index')

In [67]:
rnodes = gpd.read_file(road_folder + 'road_nodes.geojson')
rnodes = rnodes.set_index('index')
rlinks = gpd.read_file(road_folder + 'road_links.geojson')
rlinks = rlinks.set_index('index')

In [68]:
# if already mapmatched. remove road_links_list (will be redone here)
if 'road_link_list' in  links.columns:
    print('remove road_links_list')
    links = links.drop(columns = ['road_link_list'])

remove road_links_list


In [69]:
sm = stepmodel.StepModel(epsg=4326)
sm.links = links
sm.nodes = nodes
sm.road_links = rlinks
sm.road_nodes = rnodes

# preparation

In [70]:
# keep only metro.
bus = sm.copy()
bus.links = bus.links[bus.links['route_type']=='bus']

In [71]:
centroid = [*LineString(bus.nodes.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])

In [72]:
bus = bus.change_epsg(crs,coordinates_unit='meter')

Reprojecting model from epsg 4326 to epsg 32618: 100%|█| 10/10 [00:02<00:00,  3.


# mapmatching

In [73]:
bus.preparation_map_matching(sequence='link_sequence',
                             by='trip_id',
                             routing=True,
                             n_neighbors_centroid=25,
                             n_neighbors=10,
                             distance_max=3000,
                             overwrite_geom=False,
                             num_cores=num_cores)

0 / 100
0 / 100
0 / 100
0 / 100
0 / 2
2 / 2
20 / 100
20 / 100
20 / 100
20 / 100
40 / 100
40 / 100
40 / 100
40 / 100
60 / 100
60 / 100
60 / 100
80 / 100
60 / 100
80 / 100
80 / 100
100 / 100
100 / 100
100 / 100
80 / 100
100 / 100


In [74]:
# only keep road_link_list.

sm.links = sm.links.merge(bus.links[['road_link_list']],left_index=True,right_index=True,how='left')

In [75]:
sm.links['road_link_list'] = sm.links['road_link_list'].fillna('[]')

In [76]:
sm.links['road_link_list'] = sm.links['road_link_list'].astype(str)

# export

In [58]:
sm.links.to_file(pt_folder + 'links.geojson',driver='GeoJSON') 
